In [1]:
import pandas as pd

ts_data = pd.read_parquet('/Users/yashwantsaikoneru/taxi_demand_predictor/data/raw/transformed/validated_rides_2023_10.parquet')
ts_data

pickup_hour  rides  pickup_location_id
0      2023-10-01 00:00:00     62                   4
1      2023-10-01 01:00:00     73                   4
2      2023-10-01 02:00:00     61                   4
3      2023-10-01 03:00:00     32                   4
4      2023-10-01 04:00:00     12                   4
...                    ...    ...                 ...
190459 2023-10-31 19:00:00      0                 187
190460 2023-10-31 20:00:00      0                 187
190461 2023-10-31 21:00:00      0                 187
190462 2023-10-31 22:00:00      0                 187
190463 2023-10-31 23:00:00      0                 187

[190464 rows x 3 columns]

In [2]:
ts_data_one_location = ts_data.loc[ts_data.pickup_location_id == 43, :].reset_index(drop=True)
ts_data_one_location.head(15)

pickup_hour  rides  pickup_location_id
0  2023-10-01 00:00:00     24                  43
1  2023-10-01 01:00:00     14                  43
2  2023-10-01 02:00:00      6                  43
3  2023-10-01 03:00:00      0                  43
4  2023-10-01 04:00:00      3                  43
5  2023-10-01 05:00:00      3                  43
6  2023-10-01 06:00:00     14                  43
7  2023-10-01 07:00:00      8                  43
8  2023-10-01 08:00:00     26                  43
9  2023-10-01 09:00:00     51                  43
10 2023-10-01 10:00:00     86                  43
11 2023-10-01 11:00:00    110                  43
12 2023-10-01 12:00:00    148                  43
13 2023-10-01 13:00:00    136                  43
14 2023-10-01 14:00:00    143                  43

In [3]:
def get_cutoff_indices(
    data: pd.DataFrame,
    n_features: int,
    step_size: int
    ) -> list:

        stop_position = len(data) - 1
        
        # Start the first sub-sequence at index position 0
        subseq_first_idx = 0
        subseq_mid_idx = n_features
        subseq_last_idx = n_features + 1
        indices = []
        
        while subseq_last_idx <= stop_position:
            indices.append((subseq_first_idx, subseq_mid_idx, subseq_last_idx))
            
            subseq_first_idx += step_size
            subseq_mid_idx += step_size
            subseq_last_idx += step_size

        return indices

In [4]:
n_features = 24
step_size = 1

indices = get_cutoff_indices(
    ts_data_one_location,
    n_features,
    step_size
)
indices[:5]

[(0, 24, 25), (1, 25, 26), (2, 26, 27), (3, 27, 28), (4, 28, 29)]

In [5]:
import numpy as np

n_examples = len(indices)
x = np.ndarray(shape=(n_examples, n_features), dtype=np.float32)
y = np.ndarray(shape=(n_examples), dtype=np.float32)
pickup_hours = []

for i, idx in enumerate(indices):
    x[i, :] = ts_data_one_location.iloc[idx[0]:idx[1]]['rides'].values
    y[i] = ts_data_one_location.iloc[idx[1]:idx[2]]['rides'].values
    pickup_hours.append(ts_data_one_location.iloc[idx[1]]['pickup_hour'])

In [6]:
print(f'{x.shape=}')
print(f'{x=}')
print(f'{pickup_hours[:5]=}')

x.shape=(719, 24)
x=array([[ 24.,  14.,   6., ...,  30.,  29.,  10.],
       [ 14.,   6.,   0., ...,  29.,  10.,   1.],
       [  6.,   0.,   3., ...,  10.,   1.,   2.],
       ...,
       [167., 176.,  64., ..., 158., 122.,  51.],
       [176.,  64.,  21., ..., 122.,  51.,  88.],
       [ 64.,  21.,   4., ...,  51.,  88.,  49.]], dtype=float32)
pickup_hours[:5]=[Timestamp('2023-10-02 00:00:00'), Timestamp('2023-10-02 01:00:00'), Timestamp('2023-10-02 02:00:00'), Timestamp('2023-10-02 03:00:00'), Timestamp('2023-10-02 04:00:00')]


In [7]:
features_one_location = pd.DataFrame(
    x,
    columns=[f'rides_previous_{i+1}_hour' for i in reversed(range(n_features))]
)
features_one_location

rides_previous_24_hour  rides_previous_23_hour  rides_previous_22_hour  \
0                      24.0                    14.0                     6.0   
1                      14.0                     6.0                     0.0   
2                       6.0                     0.0                     3.0   
3                       0.0                     3.0                     3.0   
4                       3.0                     3.0                    14.0   
..                      ...                     ...                     ...   
714                   124.0                    88.0                   167.0   
715                    88.0                   167.0                   176.0   
716                   167.0                   176.0                    64.0   
717                   176.0                    64.0                    21.0   
718                    64.0                    21.0                     4.0   

     rides_previous_21_hour  rides_previous_20_hour  rides_previous_19_hour  \
0                       0.0                     3.0                     3.0   
1                       3.0                     3.0                    14.0   
2                       3.0                    14.0                     8.0   
3                      14.0                     8.0                    26.0   
4                       8.0                    26.0                    51.0   
..                      ...                     ...                     ...   
714                   176.0                    64.0                    21.0   
715                    64.0                    21.0                     4.0   
716                    21.0                     4.0                     4.0   
717                     4.0                     4.0                     1.0   
718                     4.0                     1.0                     0.0   

     rides_previous_18_hour  rides_previous_17_hour  rides_previous_16_hour  \
0                      14.0                     8.0                    26.0   
1                       8.0                    26.0                    51.0   
2                      26.0                    51.0                    86.0   
3                      51.0                    86.0                   110.0   
4                      86.0                   110.0                   148.0   
..                      ...                     ...                     ...   
714                     4.0                     4.0                     1.0   
715                     4.0                     1.0                     0.0   
716                     1.0                     0.0                     1.0   
717                     0.0                     1.0                     1.0   
718                     1.0                     1.0                    13.0   

     rides_previous_15_hour  ...  rides_previous_10_hour  \
0                      51.0  ...                   143.0   
1                      86.0  ...                   153.0   
2                     110.0  ...                   216.0   
3                     148.0  ...                   137.0   
4                     136.0  ...                   104.0   
..                      ...  ...                     ...   
714                     0.0  ...                    87.0   
715                     1.0  ...                    87.0   
716                     1.0  ...                    95.0   
717                    13.0  ...                   110.0   
718                    55.0  ...                   129.0   

     rides_previous_9_hour  rides_previous_8_hour  rides_previous_7_hour  \
0                    153.0                  216.0                  137.0   
1                    216.0                  137.0                  104.0   
2                    137.0                  104.0                   63.0   
3                    104.0                   63.0                   35.0   
4                     63.0                   35.0        

In [8]:
targets_one_location = pd.DataFrame(y, columns=[f'target_rides_next_hour'])
targets_one_location

target_rides_next_hour
0                       1.0
1                       2.0
2                       0.0
3                       0.0
4                       2.0
..                      ...
714                   122.0
715                    51.0
716                    88.0
717                    49.0
718                    37.0

[719 rows x 1 columns]

In [9]:
from tqdm import tqdm

def transform_ts_data_into_features_and_target(
    ts_data: pd.DataFrame,
    input_seq_len: int,
    step_size: int
) -> pd.DataFrame:
    """
    Slices and transposes data from time-series format into a (features, target)
    format that we can use to train Supervised ML models
    """
    assert set(ts_data.columns) == {'pickup_hour', 'rides', 'pickup_location_id'}

    location_ids = ts_data['pickup_location_id'].unique()
    features = pd.DataFrame()
    targets = pd.DataFrame()
    
    for location_id in tqdm(location_ids):
        
        # keep only ts data for this `location_id`
        ts_data_one_location = ts_data.loc[
            ts_data.pickup_location_id == location_id, 
            ['pickup_hour', 'rides']
        ]

        # pre-compute cutoff indices to split dataframe rows
        indices = get_cutoff_indices(
            ts_data_one_location,
            input_seq_len,
            step_size
        )

        # slice and transpose data into numpy arrays for features and targets
        n_examples = len(indices)
        x = np.ndarray(shape=(n_examples, input_seq_len), dtype=np.float32)
        y = np.ndarray(shape=(n_examples), dtype=np.float32)
        pickup_hours = []
        for i, idx in enumerate(indices):
            x[i, :] = ts_data_one_location.iloc[idx[0]:idx[1]]['rides'].values
            y[i] = ts_data_one_location.iloc[idx[1]:idx[2]]['rides'].values
            pickup_hours.append(ts_data_one_location.iloc[idx[1]]['pickup_hour'])

        # numpy -> pandas
        features_one_location = pd.DataFrame(
            x,
            columns=[f'rides_previous_{i+1}_hour' for i in reversed(range(input_seq_len))]
        )
        features_one_location['pickup_hour'] = pickup_hours
        features_one_location['pickup_location_id'] = location_id

        # numpy -> pandas
        targets_one_location = pd.DataFrame(y, columns=[f'target_rides_next_hour'])

        # concatenate results
        features = pd.concat([features, features_one_location])
        targets = pd.concat([targets, targets_one_location])

    features.reset_index(inplace=True, drop=True)
    targets.reset_index(inplace=True, drop=True)

    return features, targets['target_rides_next_hour']

In [10]:
features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*7*1, # one week of history
    step_size=24,
)

print(f'{features.shape=}')
print(f'{targets.shape=}')

  0%|          | 0/256 [00:00<?, ?it/s]

100%|██████████| 256/256 [00:01<00:00, 212.42it/s]

features.shape=(6144, 170)
targets.shape=(6144,)
